In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)nstruct-sharded/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)ded/resolve/main/configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)-sharded/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)arded/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)ruct-sharded/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)ded/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
# prompt = """
# <human>: write an email to wish everyone a happy teacher's day
# <assistant>:
# """.strip()

In [ ]:
# generation_config = model.generation_config
# generation_config.max_new_tokens = 500
# generation_config.temperature = 0.7
# generation_config.top_p = 0.7
# generation_config.num_return_sequences = 1
# generation_config.pad_token_id = tokenizer.eos_token_id
# generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# %%time
# device = "cuda:0"

# encoding = tokenizer(prompt, return_tensors="pt").to(device)
# with torch.inference_mode():
#   outputs = model.generate(
#       input_ids = encoding.input_ids,
#       attention_mask = encoding.attention_mask,
#       generation_config = generation_config
#   )

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Prep dataset

In [ ]:

import pandas as pd
from datasets import Dataset

# Load your Excel file into a pandas DataFrame
excel_file = '/content/Email_Dataset.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file)

# Convert the DataFrame to a datasets.arrow_dataset.Dataset
data = Dataset.from_pandas(df)

# Now 'dataset' is a datasets.arrow_dataset.Dataset

# You can access and manipulate the data as needed, for example:
print(data.features)  # Print dataset features
print(data[0])         # Access the first row of data

{'Prompts': Value(dtype='string', id=None), 'Generated_Email': Value(dtype='string', id=None)}
{'Prompts': "You've been invited to a job interview. Write an email confirming your attendance.", 'Generated_Email': "Subject: Confirmation of Attendance for Job Interview\n\nDear [Interviewer's Name],\n\nI hope this email finds you well. I would like to express my sincere gratitude for considering me for the [Position Name] role at [Company Name]. It is with great enthusiasm that I accept your invitation to attend the upcoming job interview scheduled for [Date] at [Location].\n\nI am genuinely excited about the opportunity to discuss my qualifications and learn more about the position and the team at [Company Name]. Rest assured, I am fully committed to the interview process and eager to demonstrate how my skills and experiences align with the requirements of the role.\n\nPlease find below a confirmation of the interview details:\n\n- Date: [Date]\n- Time: [Time]\n- Location: [Address]\n- In

In [ ]:
# data = load_dataset("csv", data_files="midjourney_prompt_dataset.csv")

In [ ]:
data

Dataset({
    features: ['Prompts', 'Generated_Email'],
    num_rows: 371
})

In [ ]:
from datasets import DatasetDict
data = DatasetDict({
    "train": data
})

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Prompts', 'Generated_Email'],
        num_rows: 371
    })
})

In [ ]:
data["train"][0]

{'Prompts': "You've been invited to a job interview. Write an email confirming your attendance.",
 'Generated_Email': "Subject: Confirmation of Attendance for Job Interview\n\nDear [Interviewer's Name],\n\nI hope this email finds you well. I would like to express my sincere gratitude for considering me for the [Position Name] role at [Company Name]. It is with great enthusiasm that I accept your invitation to attend the upcoming job interview scheduled for [Date] at [Location].\n\nI am genuinely excited about the opportunity to discuss my qualifications and learn more about the position and the team at [Company Name]. Rest assured, I am fully committed to the interview process and eager to demonstrate how my skills and experiences align with the requirements of the role.\n\nPlease find below a confirmation of the interview details:\n\n- Date: [Date]\n- Time: [Time]\n- Location: [Address]\n- Interviewers: [Interviewer Names, if available]\n\nI appreciate your consideration and look forw

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Prompts"]}
<assistant>: {data_point["Generated_Email"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/371 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Prompts', 'Generated_Email', 'input_ids', 'attention_mask'],
    num_rows: 371
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=30,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.475100
2,1.510100
3,1.506000
4,1.444500
5,1.476000
6,1.452800
7,1.519400
8,1.536300
9,1.231900
10,1.495400


TrainOutput(global_step=2760, training_loss=0.266947043774501, metrics={'train_runtime': 7710.0631, 'train_samples_per_second': 1.444, 'train_steps_per_second': 0.358, 'total_flos': 8.666501085786086e+16, 'train_loss': 0.266947043774501, 'epoch': 29.76})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
notebook_login()

In [ ]:
PEFT_MODEL = "AJ02/FineTuned-Falcon-7B"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AJ02/FineTuned-Falcon-7B/commit/91189c1477f958f0e45c5f11c091c92ab6d74a4c', commit_message='Upload model', commit_description='', oid='91189c1477f958f0e45c5f11c091c92ab6d74a4c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

(…)lcon-7B/resolve/main/adapter_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>:Write an email to apply leave for 2 days on the 18th and 19th of Noevmeber, 2023
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>:Write an email to apply leave for 2 days on the 18th and 19th of Noevmeber, 2023
<assistant>: Subject: Request for Leave from 18th to 19th December, 2023

Dear [Supervisor's Name],

I hope this email finds you well. I am writing to request permission for my absence from work from 18th to 19th of December, 2023. This is due to a personal matter that I believe requires my attention and focus over this period.

Here are the details of my leave request:

Date: 18th and 19th of December, 2023
Duration: 2 days

I have communicated this request to my team and they have agreed to accommodate my request. However, I would like to ensure that all necessary processes and arrangements are in place for my absence.

I understand that my absence will impact our work and will take proactive measures to ensure that our projects continue smoothly during this period. I am confident that my colleagues will be able to cover for my responsibilities effectively.
CPU times: user 28.4 s, sys: 56.7 ms, t